## Working with OLC in Vgrid DGGS

[![image](https://jupyterlite.rtfd.io/en/latest/_static/badge.svg)](https://demo.vgrid.vn/lab/index.html?path=vgrid/11_olc.ipynb)
[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeoshub/vgrid/blob/main/docs/notebooks/11_olc.ipynb)
[![image](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/opengeoshub/vgrid/HEAD?filepath=docs/notebooks/11_olc.ipynb)
[![image](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/opengeoshub/vgrid/blob/main/docs/notebooks/11_olc.ipynb)
[![image](https://jupyterlite.rtfd.io/en/latest/_static/badge.svg)](https://demo.gishub.vn/lab/index.html?path=notebooks/vgrid/11_olc.ipynb)

Full Vgrid DGGS documentation is available at [vgrid document](https://vgrid.gishub.vn).

To work with Vgrid DGGS directly in GeoPandas and Pandas, please use [vgridpandas](https://pypi.org/project/vgridpandas/). Full Vgridpandas DGGS documentation is available at [vgridpandas document](https://vgridpandas.gishub.vn).

To work with Vgrid DGGS in QGIS, install the [Vgrid Plugin](https://plugins.qgis.org/plugins/vgridtools/).

To visualize DGGS in Maplibre GL JS, try the [vgrid-maplibre](https://www.npmjs.com/package/vgrid-maplibre) library.

For an interactive demo, visit the [Vgrid Homepage](https://vgrid.vn).

### Install vgrid
Uncomment the following line to install [vgrid](https://pypi.org/project/vgrid/).

In [ ]:
# %pip install vgrid --upgrade

### latlon2olc

In [ ]:
from vgrid.conversion.latlon2dggs import latlon2olc

lat = 10.775276
lon = 106.706797
res = 8
olc_id = latlon2olc(lat, lon, res)
olc_id

### OLC to Shapely Polygon

In [ ]:
from vgrid.conversion.dggs2geo.olc2geo  import olc2geo

olc_geo = olc2geo(olc_id)
olc_geo

### OLC to GeoJSON

In [ ]:
from vgrid.conversion.dggs2geo.olc2geo import olc2geojson

olc_geojson = olc2geojson(olc_id)
olc_geojson

### Vector to OLC

In [ ]:
from vgrid.conversion.vector2dggs.vector2olc import vector2olc

file_path = (
    "https://raw.githubusercontent.com/opengeoshub/vopendata/main/shape/polygon2.geojson"
)
vector_to_olc = vector2olc(file_path, resolution=8, compact=False, topology = True,
                            predicate = "intersects",output_format="gpd")
# Visualize the output
vector_to_olc.plot(edgecolor="white")

### OLC Compact

In [ ]:
from vgrid.conversion.dggscompact.olccompact import olccompact

olc_compacted = olccompact(vector_to_olc, olc_id="olc", output_format="gpd")
olc_compacted.plot(edgecolor="white")

### OLC Expand

In [ ]:
from vgrid.conversion.dggscompact.olccompact import olcexpand

olc_expanded = olcexpand(vector_to_olc, resolution=10, output_format="gpd")
olc_expanded.plot()

### OLC Binning

In [ ]:
from vgrid.binning.olcbin import olcbin
import geopandas as gpd
file_path = (
    "https://raw.githubusercontent.com/opengeoshub/vopendata/main/csv/dist1_pois.csv"
)
stats = "count"
olc_bin =  olcbin(file_path, resolution=8, stats=stats, 
                # numeric_field="confidence",
                # category="category",
                output_format="gpd")
olc_bin.plot(
    column=stats,        # numeric column to base the colors on
    cmap='Spectral_r',        # color scheme (matplotlib colormap)
    legend=True,  
    linewidth=0.2          # boundary width (optional)
)   

### Raster to OLC

#### Download and open raster

In [ ]:
from vgrid.utils.io import download_file
import rasterio
from rasterio.plot import show

raster_url = (
    "https://raw.githubusercontent.com/opengeoshub/vopendata/main/raster/rgb.tif"
)
raster_file = download_file(raster_url)
src = rasterio.open(raster_file, 'r')
print(src.meta)
show(src)

#### Convert raster to OLC

In [ ]:
# %pip install folium

In [ ]:
from vgrid.conversion.raster2dggs.raster2olc    import raster2olc

raster_to_olc =  raster2olc(raster_file, output_format="gpd")

# Visualize the output
import folium

m = folium.Map(tiles="CartoDB positron", max_zoom=28)

olc_layer = folium.GeoJson(
    raster_to_olc,
    style_function=lambda x: {
        "fillColor": f"rgb({x['properties']['band_1']}, {x['properties']['band_2']}, {x['properties']['band_3']})",
        "fillOpacity": 1,
        "color": "black",
        "weight": 1,
    },
    popup=folium.GeoJsonPopup(
        fields=["olc", "band_1", "band_2", "band_3"],
        aliases=["OLC ID", "Band 1", "Band 2", "Band 3"],
        style="""
            background-color: white;
            border: 2px solid black;
            border-radius: 3px;
            box-shadow: 3px;
        """,
    ),
).add_to(m)

m.fit_bounds(olc_layer.get_bounds())

# Display the map
m

### OLC Generator

In [ ]:
from vgrid.generator.olcgrid import olcgrid
olc_grid = olcgrid(resolution=2,output_format="gpd")
# olc_grid = olcgrid(resolution=8,bbox=[106.699007, 10.762811, 106.717674, 10.778649],output_format="gpd")
olc_grid.plot(edgecolor="white")

### OLC Inspect

In [ ]:
from vgrid.stats.olcstats import olcinspect 
resolution = 4
olc_inspect = olcinspect(resolution)    
olc_inspect.head()

### OLC Normalized Area Histogram

In [ ]:
from vgrid.stats.olcstats import olc_norm_area_hist
olc_norm_area_hist(olc_inspect)

### Distribution of OLC Area Distortions

In [ ]:
from vgrid.stats.olcstats import olc_norm_area
olc_norm_area(olc_inspect)

### OLC IPQ Compactness Histogram

Isoperimetric Inequality (IPQ) Compactness (suggested by Osserman, 1987):
$$C_{IPQ} = \frac{4 \pi A}{p^2}$$
The range of the IPQ compactness metric is [0,1]. 

A circle represents the maximum compactness with a value of 1. 

As shapes become more irregular or elongated, their compactness decreases toward 0.

In [ ]:
from vgrid.stats.olcstats import olc_compactness_hist
olc_compactness_hist(olc_inspect)

### Distribution of OLC IPQ Compactness

In [ ]:
from vgrid.stats.olcstats import olc_compactness
olc_compactness(olc_inspect)

### OLC Statistics

Characteristic Length Scale (CLS - suggested by Ralph Kahn): the diameter of a spherical cap of the same cell's area

In [ ]:
from vgrid.stats import olcstats
olcstats()